In [5]:
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageMessage, AudioMessage
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [6]:
# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
        
#     # 將菜單綁定在用戶身上
#     linkRichMenuId=secretFileContentJson.get("rich_menu_id")
#     linkResult=line_bot_api.link_rich_menu_to_user(secretFileContentJson["self_user_id"], linkRichMenuId)
    
#     # 回覆文字消息與圖片消息
#     line_bot_api.reply_message(
#         event.reply_token,
#         reply_message_list
#     )

In [10]:
def PushMessage(context):
    #推播消息
    from linebot import LineBotApi
    from linebot.models import TextSendMessage
    from linebot.exceptions import LineBotApiError

    token = "AhuuqOghydZqwlCbgOHo/JJxADgxz3begLh9bzoK7nCja4ipotAl9pnKi5YAX+1eip/O65jb+4J7kdsOcVf4lViTf/RJiKymXsctgk0gS9WhD8KeNtn1WRgICzk8I3a2oQpDWx4fXcNZfcsPA1chUgdB04t89/1O/w1cDnyilFU="

    line_bot_api = LineBotApi(token)

    try:
        line_bot_api.push_message('Ubbe1a0ecaf2b210fd618c919d1d19870', TextSendMessage(text=context))
    except LineBotApiError as e:
        print("error handle")

In [2]:
from aip import AipSpeech
import json
import time
import os
import wave
from pydub import AudioSegment
from predict import *  ## * 很重要,pip install才可以直接import 自製的py檔要加*
from Voice2Text import *


#情感分析
def emotion_predict(text):
    sm = sentimentModel(text)
    #clear_output()
    model = sm.loadModel(sm.modelPath)

    print(f"要預測的話:\n{text}")
    print("="*25 + "開始預測" + "="*25)
    token_tensor, segment_tensor, mask_tensor = sm.convert_text_to_bertEat(text)
    pred_num, pred = sm.sentimentPredict(model, token_tensor, segment_tensor, mask_tensor)

    sign = sm.pass_sign_to_lineAndgui_or_nextModel(pred_num)
    
    print(f"預測結果為:{pred}")
    print("="*25 + "預測結束" + "="*25)
    
    return sign

In [3]:
@handler.add(MessageEvent, message=AudioMessage)
def handle_message(event):
    #接收資料,回傳一段訊息給用戶,並將資料取回本機
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='沒事亂傳訊息，害我又要加班'))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./audios/'+event.message.id+".aac", 'wb') as fd: ###LINE預設音檔是.aac
        for chunk in message_content.iter_content():
            fd.write(chunk)
            
    #####這邊要加入語音轉文字#####
    voice_to_txt()
    
    #####接著情感分析#####
    predict = emotion_predict(str(text))

In [7]:
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Nov/2019 20:43:06] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Nov/2019 20:43:09] "POST / HTTP/1.1" 200 -


In [24]:
# from aip import AipSpeech
# import json
# import time
# import os
# import wave
# from pydub import AudioSegment
# from predict import *  ## * 很重要,pip install才可以直接import 自製的py檔要加*

# #情感分析
# def emotion_predict(text):
#     sm = sentimentModel(text)
#     #clear_output()
#     model = sm.loadModel(sm.modelPath)

#     print(f"要預測的話:\n{text}")
#     print("="*25 + "開始預測" + "="*25)
#     token_tensor, segment_tensor, mask_tensor = sm.convert_text_to_bertEat(text)
#     pred_num, pred = sm.sentimentPredict(model, token_tensor, segment_tensor, mask_tensor)

#     sign = sm.pass_sign_to_lineAndgui_or_nextModel(pred_num)
    
#     print(f"預測結果為:{pred}")
#     print("="*25 + "預測結束" + "="*25)
    
#     return sign

# #取出判別的內文
# text = "信用卡申辦"
# print(text)
# predict = emotion_predict(str(text))

信用卡申辦


AttributeError: module 'torch' has no attribute 'tensor'

In [18]:
# from aip import AipSpeech
# import json
# import time

# """ 你的 APPID AK SK """
# APP_ID = '17610496'
# API_KEY = '3HxD7AjTwLrjEdWzjno97lbl'
# SECRET_KEY = 'rPBbzb3enl2EA0cK0n98zHktbWDqYPt8'

# begin = time.time()
# client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)


# # 读取文件
# def get_file_content(filepath):
#     with open(filepath, 'rb') as fp:
#         return fp.read()


# # 识别本地文件
# voicefile = "audios10832336465039.wav"
# # base = "./audios/"
# # print(base+voicefile)
# result = client.asr(get_file_content(voicefile), 'wav', 16000, {
#     'dev_pid': 1536,
# })
# # print(result)
# print("Request time cost %f" % (time.time() - begin))
# if result['err_no'] == 0:
#     ofile = "result_%s.txt" % (voicefile)
#     with open(ofile, "w", encoding="utf-8") as of:
#         json.dump(result, of, ensure_ascii=False)
# else:
#     print(result['err_msg'], result['err_no'])

# print("完成了")

In [19]:
# import wave

# with wave.open('./audios/10826265961358.wav', "rb") as f:
#     f = wave.open('./audios/10826265961358.wav')
#     print(f.getparams())

In [20]:
# import librosa
# import soundfile as sf
# x,_ = librosa.load('./audios/10826265961358.wav', sr=16000)
# sf.write('tmp.wav', x, 16000)
# wave.open('tmp.wav','r')

In [21]:
# import os
# import wave
# os.system(r"ffmpeg -i 10826265961358.wav b0.wav")
# wave.open('b0', "r")

In [22]:
# b=os.popen(r"ffmpeg -y -i ./audios/10826265961358.wav  -acodec pcm_s16le -f s16le -ac 1 -ar 16000 ./audios/1082626596135800.wav")

In [23]:
# from google.colab import drive